# Bigram Language Model
The simplest baseline model for text generation.

In [6]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'

## 1. Dataset

In [7]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [8]:
print(len(text))
print(text[:100])

1115393
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


## 2. Tokenization, train/val split

In [9]:
# create all chars (as vocabulary/token)
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(vocab_size, ''.join(chars))

65 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [10]:
# lookup table
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
# simple tokenizer
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

tokenized = encode('hello there')
print(tokenized)
print(decode(tokenized))

[46, 43, 50, 50, 53, 1, 58, 46, 43, 56, 43]
hello there


In [11]:
# load all the tokenized dataset into tensor
data = torch.tensor(encode(text), dtype=torch.long) # todo use gpu?
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115393]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [12]:
# train/val split
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

## 3. Data loader

In [13]:
context_length: int = 8

In [14]:
x = train_data[:context_length]
y = train_data[1:context_length+1]
for t in range(context_length): # time dimension
    context = x[:t+1]
    target = y[t]
    print(context, ':', target)


tensor([18]) : tensor(47)
tensor([18, 47]) : tensor(56)
tensor([18, 47, 56]) : tensor(57)
tensor([18, 47, 56, 57]) : tensor(58)
tensor([18, 47, 56, 57, 58]) : tensor(1)
tensor([18, 47, 56, 57, 58,  1]) : tensor(15)
tensor([18, 47, 56, 57, 58,  1, 15]) : tensor(47)
tensor([18, 47, 56, 57, 58,  1, 15, 47]) : tensor(58)


In [15]:

torch.manual_seed(42)
def get_batch(split: str, batch_size: int = 4) -> tuple[torch.Tensor, torch.Tensor]:
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - context_length, (batch_size,))
    x: torch.Tensor = torch.stack([data[i:i+context_length] for i in ix])
    y: torch.Tensor = torch.stack([data[i+1:i+context_length+1] for i in ix])
    return x.to(device), y.to(device)

xb, yb = get_batch('train')
print('inputs:', xb)
print('targets:', yb)

inputs: tensor([[54, 43, 39, 49,  1, 39, 45, 39],
        [43, 56, 57, 11,  1, 61, 46, 53],
        [43,  1, 46, 39, 58, 46,  1, 58],
        [43,  1, 47, 57,  1, 57, 53,  1]], device='cuda:0')
targets: tensor([[43, 39, 49,  1, 39, 45, 39, 47],
        [56, 57, 11,  1, 61, 46, 53,  6],
        [ 1, 46, 39, 58, 46,  1, 58, 56],
        [ 1, 47, 57,  1, 57, 53,  1, 50]], device='cuda:0')


## 4. Simplest Baseline (BigramLanguageModel)

In [16]:
from typing import Optional

import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(42)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size: int):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table: nn.Embedding = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx: torch.Tensor, targets: Optional[torch.Tensor] = None) -> tuple[torch.Tensor, Optional[torch.Tensor]]:
        # idx and targets are both (B,T)
        logits = self.token_embedding_table(idx) # (B,T,C), C -> vocab_size

        # loss function using negative log likelihood loss
        loss = None
        if targets is not None:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx: torch.Tensor, max_new_tokens: int):
        # idx is (B,T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step (get the last column only in the T dimension)
            logits = logits[:, -1, :]   # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)   # get the probabilities of C dimension (last dimension)
            # sample from the distribution using probabilities
            idx_next = torch.multinomial(probs, num_samples=1)  # will give us (B, 1)
            # append the sampled idx (the predicted) to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx
    
m = BigramLanguageModel(vocab_size=vocab_size).to(device)
out, loss = m(xb, yb)
print(out.shape)
print(loss)

torch.Size([32, 65])
tensor(4.7187, device='cuda:0', grad_fn=<NllLossBackward0>)


In [17]:
idx = torch.zeros((1, 1), dtype=torch.long).to(device) # 0 is 'newline' character in our vocab
logits = m.generate(idx, max_new_tokens=100)[0].tolist()
decoded_logits = decode(logits)
print(decoded_logits)


J
3u :Y3RJtbzDP'VFTwPGt'uY3UWdXR3LnqTCU,XptxcGY,uOhjZPImXEP-MAS-xKpGpOwisCbMI;IT3sDqrgZPptbAstPJ.WGI


In [18]:
# create optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [19]:
@torch.no_grad()
def estimate_loss(model, eval_iters: int, batch_size: int):
    out = {}
    model.eval()    # toggle the model to evaluation mode
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split, batch_size)
            _, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()   # toggle back to train mode
    return out

### Training

In [20]:
batch_size = 32
epoch_size = 5000
eval_interval = 200
eval_iters = 200
for step in range(epoch_size):
    # every once in a while, evaluate the loss on train and val sets
    if step % eval_interval == 0:
        losses = estimate_loss(m, eval_iters, batch_size)
        print(f"steps {step}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch from the train data
    xb, yb = get_batch('train', batch_size)

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)   # reset the gradient from the previous step (if any)
    loss.backward() # using the gradient from all the parameters
    optimizer.step()    # using gradient to update the parameters


steps 0: train loss 4.7541, val loss 4.7638
steps 200: train loss 4.5256, val loss 4.5448
steps 400: train loss 4.3208, val loss 4.3268
steps 600: train loss 4.1198, val loss 4.1347
steps 800: train loss 3.9395, val loss 3.9609
steps 1000: train loss 3.7779, val loss 3.7918
steps 1200: train loss 3.6295, val loss 3.6567
steps 1400: train loss 3.5034, val loss 3.5116
steps 1600: train loss 3.3740, val loss 3.3992
steps 1800: train loss 3.2639, val loss 3.3004
steps 2000: train loss 3.1722, val loss 3.1999
steps 2200: train loss 3.0830, val loss 3.1078
steps 2400: train loss 3.0042, val loss 3.0224
steps 2600: train loss 2.9456, val loss 2.9669
steps 2800: train loss 2.8803, val loss 2.9065
steps 3000: train loss 2.8231, val loss 2.8584
steps 3200: train loss 2.7766, val loss 2.8009
steps 3400: train loss 2.7383, val loss 2.7610
steps 3600: train loss 2.7092, val loss 2.7233
steps 3800: train loss 2.6734, val loss 2.6993
steps 4000: train loss 2.6504, val loss 2.6746
steps 4200: train lo

In [21]:
idx = torch.zeros((1, 1), dtype=torch.long).to(device) # 0 is 'newline' character in our vocab
logits = m.generate(idx, max_new_tokens=500)[0].tolist()
decoded_logits = decode(logits)
print(decoded_logits)


coit;inde e
WR:
PEngemangEENigesps
b&hind.
UCENGF url hathapr me m:chan fHe.
Fju d iere-ve'durore cense wn ce m3Haroqus t avest t, metha'? m; hy?



INSThyourBRGLONG;f ind s hevalshe, ENVzers ndowe:
LUSGo be'sblore ,
TZTd phSe
pefoue t he ososour talunHI'llbr!g y HoJP:
 tNEOMan te s mire akns t
Tou'esa amnfod bs as Y w:
Ked DUMIasMTVQ on myoferisthatessotrilorule, maripake 'seUCKIOW, gZdut wntide.
AS:
GSorud ve y ube I thellu f:
DWARxDZHayshapy d tr ,
Sthong-mar;UPYndd:
AVar;.
PrUCave SLyorves,



In [ ]:
### dummy self-attention (using bag of words, means of prev tokens + current token) instead of actual self-attenton mechanism
torch.manual_seed(42)
B,T,C = 4, 8, 2 # batch, time, channels
x = torch.randn((B, T, C))

# version 1
# xbow = torch.zeros((B, T, C))
# for b in range(B):
#     for t in range(T):
#         xprev = x[b, :t+1]
#         xbow[b, t] = torch.mean(xprev, dim=0)
# xbow

# version 2 (using triangular matrices trick)
# wei = torch.tril(torch.ones((T, T)))
# wei = wei / torch.sum(wei, dim=1, keepdim=True)
# xbow = wei @ x  # (T, T) @ (B, T, C) --> (B, T, T) @ (B, T, C) = (B, T, C)
# xbow

# version 3 (using masked fill with softmax trick)
tril = torch.tril(torch.ones((T, T)))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)    # for last dimension
xbow = wei @ x
xbow

tensor([[[ 1.9269,  1.4873],
         [ 1.4138, -0.3091],
         [ 1.1687, -0.6176],
         [ 0.8657, -0.8644],
         [ 0.5422, -0.3617],
         [ 0.3864, -0.5354],
         [ 0.2272, -0.5388],
         [ 0.1027, -0.3762]],

        [[ 1.6423, -0.1596],
         [ 0.5725,  0.1400],
         [ 0.1289,  0.4528],
         [ 0.2969,  0.7597],
         [ 0.4933,  0.8671],
         [ 0.5129,  0.9450],
         [ 0.4065,  0.8160],
         [ 0.3242,  0.8215]],

        [[-1.3847, -0.8712],
         [-0.8040,  0.4231],
         [-0.4297,  0.1405],
         [-0.2459, -0.0882],
         [-0.5082,  0.1285],
         [-0.5701,  0.0069],
         [-0.6707,  0.3092],
         [-0.7412,  0.2095]],

        [[-0.9138, -0.6581],
         [-0.4179, -0.0662],
         [-0.4413,  0.3530],
         [-0.5344,  0.0808],
         [-0.7082,  0.0718],
         [-0.6008,  0.1724],
         [-0.5289,  0.4113],
         [-0.6109,  0.5329]]])